## Extract amplicons from mitochondrial genomes of Anopheles

Alignment obtained as follows:

- Find 'Anopheles complete mitochondrion' in GenBank - ~130 records
- apply filters length=14000-17000, compartment='mitochondrion', exclude RefSeq (as these are duplicates) - 92 records
- align with MAFFT
- exclude CM003708 (wrong decircularization, Agam), MG930836-60 (shorter subsequence reconstructed, Agam), DQ146364 (too many Ns - the only funestus in dataset, unfortunately) - 62 records
- align with MAFFT

This notebook covers the following:

- get long amplicons (with variable insert) from mt alignment (see 20180223)
- extend variable inserts, phylogenetic summary (see 20180309)
- gene annotations (see 20180327)

In [1]:
# check biopython version - need >=1.69 to support MAF
import Bio
Bio.__version__

'1.70'

In [89]:
# Setting up - slightly modified from 20180223, 20180309, 
from Bio import AlignIO
import numpy as np
import pandas as pd
from Bio import Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator


# Alignment filtering parameters
min_aligned = 190 # minimum alignment length, also used as minimum amplicon length
min_conserved = 50 # minimum length of flanks with given conservation level - used for primer design and, if possible, target sites
max_xs = 0.1 # maximum proportion of indels (represented as X) in flanks
max_ns = 0.1 # maximum proportion of substitutions (represented as N) in flanks
max_insert = 100 # maximum length of non-conserved sequence between two conserved flanks 

def seq_repr(alignment):
    '''
    Given multiple sequence alignment, return first sequence with Ns for ambiguous chars and X's for indels.'''
    seq = ''
    for i in range(alignment.get_alignment_length()):
        col = alignment[:, i]
        if '-' in col: # indel stronger than substitution
            seq += 'X'
        elif len(set(col)) == 1:
            seq += col[0]
        else:
            seq += 'N'
    return seq

def get_conserved_subsequences(seq, max_ns=0.1, max_xs=0.1, min_len=100):
    '''
    Given sequence, conservation (max_ns) and indel (max_xs) levels, and minimum subsequence length
    return list of tuples for the subsequences with given conservation level (overlapping regions merged).
    If no conserved subsequences found, return 'None'.'''
    slen = len(seq)
    if slen < min_len:
        return None
    
    def is_conserved(s, max_ns, max_xs):
        if s.count('N')/len(s) <= max_ns and s.count('X')/len(s) <= max_xs:
            return True
        else:
            return False
    cons_windows = [is_conserved(seq[i:i + min_len], max_ns, max_xs) for i in range(slen - min_len + 1)]
    if sum(cons_windows) == 0:
        return None
    
    cons_kernels = []
    in_kernel = False
    for i, cw in enumerate(cons_windows):
        if in_kernel:
            if cw == False:
                in_kernel = False
                cons_kernels.append(i + min_len)
        elif cw == True:
            cons_kernels.append(i)
            in_kernel = True
    if in_kernel:
        cons_kernels.append(i + min_len) 
        
    # merge overlapping kernels
    merged_kernels = []
    for i in range(len(cons_kernels)//2):
        start = cons_kernels[i * 2]
        end = cons_kernels[i * 2 + 1]
        if not merged_kernels:
            merged_kernels.append((start, end))
        else:
            prev_start = merged_kernels[-1][0]
            prev_end = merged_kernels[-1][1]
            if prev_end >= start:
                upper_bound = max(prev_end, end)
                merged_kernels[-1] = (prev_start, upper_bound)  # replace by merged interval
            else:
                merged_kernels.append((start, end))
    
    return np.asarray(merged_kernels)

# Candidate amplicon search - within conserved sequences and between consecutive conserved sequences
def get_candidate_amplicons(cons, min_len=190, max_insert=50):
    '''
    Given conservation intervals, minimum amplicon lenght and maximum insert length,
    return np.array of plausible amplicons with insert positions'''
    ampls = []
    for reg in cons: # internal amplicons
        if reg[1] - reg[0] >= min_len:
            ampls.append((reg[0], reg[1], 0, 0))
    for i in range(len(cons) - 1):
        for j in range(i + 1, len(cons)):
            if cons[j, 0] - cons[i, 1] <= max_insert:
                if cons[j, 1] - cons[i, 0] >= min_len:
                    ampls.append((cons[i, 0], cons[j, 1],
                                  cons[i, 1], cons[j, 0]))
    return ampls

def gapped_coord(aln, coord, ref=0):
    '''
    Transforms coordinate in maf alignment according to number of gaps in ref (i-th seq in alignment)
    '''
    ngaps = str(aln[ref, :coord].seq).count('-')
    return coord - ngaps

def alignment_to_amplicons(alignment, min_species, min_aligned, max_xs, max_ns, min_conserved, max_insert):
    '''
    Given alignment and filtering paramters
    return list of (alignment, target start, target end)
    '''
    ampl_data = []
    if len(alignment) >= min_species and alignment.get_alignment_length() >= min_aligned:
        seq = seq_repr(alignment)
        cons = get_conserved_subsequences(seq, max_ns=max_ns, max_xs=max_xs, min_len=min_conserved)
        if cons is not None:
            ampls = get_candidate_amplicons(cons, min_aligned, max_insert)
            if len(ampls) > 0:
                for ampl in ampls:
                    ampl_aln = alignment[:, ampl[0]:ampl[1]]
                    ampl_data.append((ampl_aln, ampl))
                return ampl_data
    return None

def prop_var(seq):
    '''
    Return propotion of variable nucleotides in seq_repr of alignment'''
    return (seq.count('N') + seq.count('X'))/len(seq)

def extend_variable(seq, start, end, min_ambig=0.5):
    '''
    Extends flanks of variable insert. Works only if seq[0:start] and seq[end:len(seq)] are conserved.
    This should be true for pre-selected amplicons (see 20180223).
    Parameters - sequence, start and end of variable target to be extended,
    minimum proportion of variable sites for extended region. '''
    var_start = False
    for i in range(0, start - 1):
        if prop_var(seq[i:start]) >= min_ambig:
            #print(seq[i:start])
            var_start = True
        if var_start:
            if seq[i] in 'NX':
                ext_start = i
                #print(ext_start)
                break
    else:
        ext_start = start
    
    var_end = False
    for i in reversed(range(end + 1,len(seq))):
        if prop_var(seq[end:i]) >= min_ambig:
            #print(seq[end:i])
            var_end = True
        if var_end:
            if seq[i - 1] in 'NX':
                ext_end = i
                #print(ext_end)
                break
    else:
        ext_end = end
    
    return (ext_start, ext_end)

def identical_clusters(x):
    aln = x['aln']
    '''
    Given alignment, return list of sets with species IDs with identical sequences'''
    

    ids = [set()]
    dm = DistanceCalculator('identity').get_distance(aln)
    dm.names = [n.split('.')[0] for n in dm.names]
    for i in range(len(dm)):
        for j in range(i + 1, len(dm)):
            if dm[i,j] == 0:
                n1 = dm.names[i]
                n2 = dm.names[j]
                for cl in ids:
                    if (n1 in cl):
                        if (n2 in cl):
                            break
                        if (n2 not in cl):
                            cl.add(n2)
                            break
                else:
                    ids.append(set((n1, n2)))
        
    return ids[1:]

def annotate(x):

    ins_str = '{}\t{}\t{}\n'.format('KT382817.1', x['ins_start'], x['ins_end'])
    ins_bed = pybedtools.BedTool(ins_str, from_string=True)
    ins_annot = KT382817_annot.intersect(ins_bed).to_dataframe()
    ins_annot = ins_annot[ins_annot.feature.isin(['tRNA','rRNA','CDS'])]
    prods = []
    for x in ins_annot.attributes:
        attrs = x.split(';')
        for attr in attrs:
            if attr.startswith('product='):
                prods.append(attr[8:])
    return ';'.join(prods)

def phylo_tree(aln, vectorized=True):
    if vectorized:
        aln = aln['aln']
    '''
    Given alignment, return NJ tree in nwk format'''
    from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
    calculator = DistanceCalculator('identity')
    dm = calculator.get_distance(aln)     
    dm.names = [n.split('.')[0] for n in dm.names]
    constructor = DistanceTreeConstructor()
    tree = constructor.nj(dm)
    return tree

In [3]:
# Find amplicons in mitochondrial alignment

aln_file = '/Users/am60/malaria/20180129_phylo_ampliseq/mt_rrna/wmt65.aln.fasta'
mt_aln = AlignIO.read(aln_file, "fasta")
mt_aln
# amplicon_data = alignment_to_amplicons(mt_aln, 65, min_aligned, max_xs, max_ns, min_conserved, max_insert)
# amplicon_data

<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 16226, SingleLetterAlphabet()) at 109ce3080>

In [21]:
aln = alignment_to_amplicons(mt_aln, min_species=65, min_aligned=min_aligned, max_xs=max_xs, 
                           max_ns=max_ns, min_conserved=min_conserved, 
                           max_insert=max_insert)
aln

[(<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 200, SingleLetterAlphabet()) at 1115c16d8>,
  (5961, 6161, 0, 0)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 228, SingleLetterAlphabet()) at 111384ac8>,
  (3733, 3961, 3821, 3908)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 263, SingleLetterAlphabet()) at 111623710>,
  (5961, 6224, 6161, 6171)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 364, SingleLetterAlphabet()) at 111607438>,
  (5961, 6325, 6161, 6251)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 225, SingleLetterAlphabet()) at 11165e160>,
  (12687, 12912, 12763, 12783)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 278, SingleLetterAlphabet()) at 111644e48>,
  (12783, 13061, 12912, 12928)),
 (<<class 'Bio.Align.MultipleSeqAlignment'> instance (65 records of length 309, SingleLetterAlphabet()) at 111

In [11]:
# write only amplicons with long insert
count = 0
with open("/Users/am60/malaria/20180129_phylo_ampliseq/mt_rrna/20180414_mt_amplicons.maf", "w") as handle:
    for a in aln[1:]: 
        count += AlignIO.write(a[0], handle, "maf")
count

10

In [40]:
# extend variable inserts for long insert amplicons
# and create basic data object
long_alns = OrderedDict()
for (i, a) in enumerate(aln[1:]):
    seq = seq_repr(a[0])
    # for insert extension use internal coordinates
    (start, end) = extend_variable(seq, a[1][2]-a[1][0], a[1][3]-a[1][0])
    insert = seq[start:end]
    long_alns['mt' + str(i)] = dict([
            ('aln', a[0]), # alignment object for amplicon
            ('ampl_aln_start', a[1][0]), # coordinates in alignment
            ('ampl_aln_end', a[1][1]), 
            ('ins_aln_start', a[1][0] + start),
            ('ins_aln_end', a[1][0] + end),
            ('ampl_start', gapped_coord(mt_aln, a[1][0], ref=0)), # coordinates in KT382817.1 (Anopheles atroparvus mtDNA)
            ('ampl_start', gapped_coord(mt_aln, a[1][1], ref=0)), 
            ('ins_start', gapped_coord(mt_aln, a[1][0] + start, ref=0)),
            ('ins_end', gapped_coord(mt_aln, a[1][0] + end, ref=0)),
            ('ampl_seq', seq),
            ('ins_seq', insert)
            ])

laln_data = pd.DataFrame.from_dict(long_alns, orient='index')
laln_data[['ampl_aln_start']]

mt0     3733
mt1     5961
mt2     5961
mt3    12687
mt4    12783
mt5    12928
mt6    13156
mt7    14162
mt8    14365
mt9    14619
Name: ampl_aln_start, dtype: int64

In [48]:
# sets of sequences that cannot be identified
laln_data['identical_clusters'] = laln_data.apply(identical_clusters, axis=1)

In [52]:
import pybedtools

In [87]:
# annotate using first of sequences in 
KT382817_annot = pybedtools.BedTool('/Users/am60/malaria/20180129_phylo_ampliseq/mt_rrna/KT382817.gff3')

laln_data['ins_genes'] = laln_data.apply(annotate, axis=1)
laln_data['ins_genes'] 

mt0    tRNA-Lys;tRNA-Asp
mt1             tRNA-Asn
mt2    tRNA-Asn;tRNA-Ser
mt3    16S ribosomal RNA
mt4    16S ribosomal RNA
mt5    16S ribosomal RNA
mt6    16S ribosomal RNA
mt7    12S ribosomal RNA
mt8    12S ribosomal RNA
mt9    12S ribosomal RNA
Name: ins_genes, dtype: object

In [94]:
# add conventional marker coordinates in mt_aln
# Sallum 2002 COI - based on AF417695.1 Aalb sequence lookup in mt_aln
sallum_coi = (2069, 2971)
# Sallum 2002 COII - based on AF417731.1 Aalb sequence
sallum_coii = (3087, 3689)
# whole mitochondrial genome prior to tandem repeats
wmt = (1, 14949)

In [90]:
# construct trees for amplicons
laln_data['trees'] = laln_data.apply(phylo_tree, axis=1)

In [101]:
# construct trees for conventional markers
# write to file
# order: wmt, coi, coii, amplicons
with open('/Users/am60/malaria/20180129_phylo_ampliseq/mt_rrna/20180414_mt_trees.nwk', 'w') as o:
    for reg in (wmt, sallum_coi, sallum_coii):
        subaln = mt_aln[:, reg[0] - 1:reg[1] + 1] # first and last nucleotides of region to be included
        t = phylo_tree(subaln, vectorized=False)
        Phylo.write(t, o, 'newick')
    for t in laln_data.trees:
        Phylo.write(t, o, 'newick')
    

SingleLetterAlphabet() alignment with 65 rows and 14950 columns
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt KT382817.1/1-15458
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt L04272.1/1-15455
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt KT218684.1/1-15076
aatgaattgcctg-atgaaaaggattaccttgatagggtaaatc...tgt GQ918273.1/1-15385
aatgaattgcctg-atgaaaaggattaccttgatagggtaaatc...tgt GQ918272.1/1-15386
aatgaattgcctg-acaaaaaggattaccttgatagggtaaatc...tgt KR732656.1/1-15330
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt KT895423.1/1-15395
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt JX219736.1/1-15412
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt JX219735.1/1-15412
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt JX219734.1/1-15336
aatggattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt JX219733.1/1-15412
aatgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt KT899887.1/1-15406
-atgaattgcctg-ataaaaaggattaccttgatagggtaaatc...tgt MG930894.1/1-15364
-atgaattgcctg-ataaaaaggattac

In [113]:
# make trees for fake half-overlapping 220-bp regions from COI and COII

def split_reg(reg, l=220, s=110):
    subreg = []
    pos = reg[0] + l
    while pos < reg[1]:
        subreg.append((pos - l, pos))
        pos += s
    return subreg

with open('/Users/am60/malaria/20180129_phylo_ampliseq/mt_rrna/20180414_coi220_trees.nwk', 'w') as o:
    for reg in (sallum_coi, sallum_coii):
        srs = split_reg(reg, l=220, s=110)
        print(len(srs))
        for sr in srs:
            saln = mt_aln[:, sr[0]:sr[1]]
            st = phylo_tree(saln, vectorized=False)
            Phylo.write(st, o, 'newick')


7
4
